In [0]:
import pypdf 
pypdf.__version__

In [0]:
import mlflow
from mlflow.tracking import MlflowClient

In [0]:
def get_latest_model_version(func):
    """
    Helper method to programmatically get latest model's version from the registry
    """
    def wrapper(model_name):
        """
        Wrapper to fetch the latest model version before executing the main function.
        """
        client = MlflowClient()
        model_version_infos = client.search_model_versions("name = '%s'" % model_name)
        latest_version = max(
            [model_version_info.version for model_version_info in model_version_infos]
        )
        # Call the original function with the latest version
        return func(model_name, latest_version)

    return wrapper


@get_latest_model_version
def set_alias(model_name, current_model_version):
    # Set @alias to the latest model version
    mlflow.tracking.MlflowClient().set_registered_model_alias(
        name=model_name, alias="champion",
        version=current_model_version
        )
    
    return current_model_version

In [0]:
current_model_version = set_alias(model_name="hemzai.gold.hemz-pilot-chain")
model_uri = f"models:/hemzai.gold.hemz-pilot-chain/{current_model_version}"

In [0]:
model_uri

In [0]:
db_host = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().get("browserHostName").value()

In [0]:
!databricks secrets create-scope "hemzai_scope"

In [0]:
!databricks secrets put-secret --json '{"scope": "hemzai_scope","key": "hemzai_depl_host","string_value": "dbc-434c5869-d401.cloud.databricks.com"}'